In [8]:
import os

In [9]:
%pwd


'c:\\Users\\91810\\project\\flight\\end_to_end_flight-price-prediction\\research'

In [10]:
os.chdir("../")


In [11]:
%pwd


'c:\\Users\\91810\\project\\flight\\end_to_end_flight-price-prediction'

In [12]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [13]:
from mlProject.constants import *  # here i call all constant file path
from mlProject.utils.common import read_yaml, create_directories   # using utils i read that file and create directory

In [14]:
#create a configure manager class where a read the config and params and schema file and create a directory
#this config manager give us all the configure  we need to build the end to end project
#05 update the configuration manager in src config

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [16]:
#this is my data ingestion class here i get all the config
#06 update the components 
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


#this method download the data from the url     
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



#this method extract the zip file     
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
#07 pipeline 

try:
    config = ConfigurationManager()                                  # instance create of config manager class 
    data_ingestion_config = config.get_data_ingestion_config()       #call the dataingestion method and return all the usefull configuration
    data_ingestion = DataIngestion(config=data_ingestion_config)     #using that config we create another instance of DataIngestion class
    data_ingestion.download_file()                                   #download the data 
    data_ingestion.extract_zip_file()                                #extract the zip file
except Exception as e:
    raise e

[2024-04-17 23:39:58,789: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-17 23:39:58,797: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-17 23:39:58,808: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-17 23:39:58,809: INFO: common: created directory at: artifacts]
[2024-04-17 23:39:58,811: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-17 23:40:00,739: INFO: 3684673689: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 138397
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "8642067eb3b49c1da3b711f4177ef08a62664f6b26c4048901028919d42216e7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 19F6:73D46:32FA99:412382:6620107A
Accept-Ranges: bytes
Date: Wed, 17